In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from flarestack import analyse, ResultsHandler
from flarestack.data.simulate.potemkin import potemkin_dataset, IdealBackgroundFluxModel
from flarestack.utils.prepare_catalogue import ps_catalogue_name
from flarestack.shared import flux_to_k
import logging
logging.getLogger().setLevel("INFO")

Saving converted numpy array to /Users/robertstein/flarestack__data/input/median_angular_resolution/all_sky_3_year/IC79-2010.pkl
Saving converted numpy array to /Users/robertstein/flarestack__data/input/median_angular_resolution/all_sky_3_year/IC86-2011.pkl
Saving converted numpy array to /Users/robertstein/flarestack__data/input/median_angular_resolution/all_sky_3_year/IC86-2012.pkl
Saving to /Users/robertstein/flarestack__data/output/dataset_plots/energy_proxy_map/all_sky_3_year/IC79-2010-Upgoing.pdf
Saving converted numpy array to /Users/robertstein/flarestack__data/input/energy_proxy_weighting/all_sky_3_year/IC79-2010.pkl
Saving to /Users/robertstein/flarestack__data/output/dataset_plots/effective_area_plots/all_sky_3_year/IC79-2010.pdf
Saving to /Users/robertstein/flarestack__data/output/dataset_plots/energy_proxy_map/all_sky_3_year/IC86-2011-Upgoing.pdf
Saving converted numpy array to /Users/robertstein/flarestack__data/input/energy_proxy_weighting/all_sky_3_year/IC86-2011.pkl
Sa

In [2]:
# We create an imaginary dataset season, between 50000 MJD and 55000 MJD, with 100% uptime

bkg_time_pdf_dict = {
    "time_pdf_name": "fixed_ref_box",
    "fixed_ref_time_mjd": 50000,
    "pre_window": 0.,
    "post_window": 500.,
    "livetime": True
}

# We take the rough normalisation of the atmospheric flux, ~10^8 GeV cm^-2 s^-1 sr^-1 at ~200 TeV

f_200tev = (10. ** -8.)
flux_norm_1_gev = f_200tev * ((2.e5) ** 1.7)

bkg_flux_model = IdealBackgroundFluxModel(
    flux_norm_1_gev,
    bkg_time_pdf_dict=bkg_time_pdf_dict,
)

# We create a simulated dataset, containg only background, using the published effective areas for IceCube with 86 strings 2012

potemkin_dataset.set_sim_params(
    name="IC86-2012",
    bkg_flux_model=bkg_flux_model,
#     resimulate=True
)

INFO:root:Minimum Energy is 100.0 GeV.
INFO:root:Maximum Energy is 10000000.0 GeV
INFO:root:Simulating events:
INFO:root:Simulated 3599 events between sin(dec)=0.020408163265306145 and sin(dec)=0.06122448979591821
INFO:root:Simulated 4590 events between sin(dec)=0.06122448979591821 and sin(dec)=0.1020408163265305
INFO:root:Simulated 5628 events between sin(dec)=0.1020408163265305 and sin(dec)=0.1428571428571428
INFO:root:Simulated 6457 events between sin(dec)=0.1428571428571428 and sin(dec)=0.18367346938775508
INFO:root:Simulated 7604 events between sin(dec)=0.18367346938775508 and sin(dec)=0.22448979591836715
INFO:root:Simulated 8280 events between sin(dec)=0.22448979591836715 and sin(dec)=0.26530612244897944
INFO:root:Simulated 9050 events between sin(dec)=0.26530612244897944 and sin(dec)=0.30612244897959173
INFO:root:Simulated 9206 events between sin(dec)=0.30612244897959173 and sin(dec)=0.346938775510204
INFO:root:Simulated 9576 events between sin(dec)=0.346938775510204 and sin(dec

In [3]:
# Setup analysis, injecting an E^-2 source

inj_dict = {
    "injection_sig_time_pdf": {
        "time_pdf_name": "steady"
    },
    "injection_energy_pdf": {
        "energy_pdf_name": "power_law",
        "gamma": 2.0
    }
}

# Perform a "standard" analysis (fitting spectral index and ns)

llh_dict = {
    "llh_name": "standard",
    "llh_sig_time_pdf": {
        "time_pdf_name": "steady"
    },
    "llh_bkg_time_pdf":  {
        "time_pdf_name": "steady"
    },
    "llh_energy_pdf": {
        "energy_pdf_name": "power_law"
    }
}

# Analysis with "source catalogue" consisting of a single source at sindec=0.5

mh_dict = {
            "name": "examples/analysis_with_sim",
            "mh_name": "fixed_weights",
            "dataset": potemkin_dataset.get_seasons("IC86-2012"),
            "catalogue": ps_catalogue_name(0.5),
            "inj_dict": inj_dict,
            "llh_dict": llh_dict,
            "n_steps": 5,
            "n_trials": 30,
            "scale": 100.
        }

INFO:root:Saving to /Users/robertstein/flarestack__data/input/catalogues/single_source/sindec_0.50.npy


In [4]:
print(potemkin_dataset.get_single_season("IC86-2012"))

In [5]:
analyse(mh_dict, n_cpu=2, cluster=False)

rh = ResultsHandler(mh_dict)

INFO:root:Using 'standard' LLH class
INFO:root:Initialising Injector for IC86-2012
INFO:root:Saving to
INFO:root:Using 'standard' LLH class
INFO:root:Using 'standard' LLH class
INFO:root:Added 420 trials to queue. Now processing.
INFO: 2020-05-10 13:58:32,538 - 88028 - Using 'standard' LLH class
INFO:root:420 tasks remaining.
INFO: 2020-05-10 13:58:32,542 - 88029 - Using 'standard' LLH class
INFO:root:Saving IC86-2012 acceptance values to: /Users/robertstein/flarestack__data/input/acceptance_functions/SimCube_default/IC86-2012.pkl
INFO: 2020-05-10 13:58:33,664 - 88029 - Saving IC86-2012 acceptance values to: /Users/robertstein/flarestack__data/input/acceptance_functions/SimCube_default/IC86-2012.pkl
INFO:root:Saving IC86-2012 acceptance values to: /Users/robertstein/flarestack__data/input/acceptance_functions/SimCube_default/IC86-2012.pkl
INFO: 2020-05-10 13:58:33,671 - 88028 - Saving IC86-2012 acceptance values to: /Users/robertstein/flarestack__data/input/acceptance_functions/SimCube